In [ ]:
# The 6.00 Word Game

import random
import string

VOWELS = 'aeiou'
CONSONANTS = 'bcdfghjklmnpqrstvwxyz'
HAND_SIZE = 7

SCRABBLE_LETTER_VALUES = {
    'a': 1, 'b': 3, 'c': 3, 'd': 2, 'e': 1, 'f': 4, 'g': 2, 'h': 4, 'i': 1, 'j': 8, 'k': 5, 'l': 1, 'm': 3, 'n': 1,
    'o': 1, 'p': 3, 'q': 10, 'r': 1, 's': 1, 't': 1, 'u': 1, 'v': 4, 'w': 4, 'x': 8, 'y': 4, 'z': 10
}

# -----------------------------------
# Helper code
# (you don't need to understand this helper code)

WORDLIST_FILENAME = "C:/Education/data/words.txt"

def loadWords():
    """
    Returns a list of valid words. Words are strings of lowercase letters.
    
    Depending on the size of the word list, this function may
    take a while to finish.
    """
    print("Loading word list from file...")
    # inFile: file
    inFile = open(WORDLIST_FILENAME, 'r')
    # wordList: list of strings
    wordList = []
    for line in inFile:
        wordList.append(line.strip().lower())
    print("  ", len(wordList), "words loaded.")
    return wordList

def getFrequencyDict(sequence):
    """
    Returns a dictionary where the keys are elements of the sequence
    and the values are integer counts, for the number of times that
    an element is repeated in the sequence.

    sequence: string or list
    return: dictionary
    """
    # freqs: dictionary (element_type -> int)
    freq = {}
    for x in sequence:
        freq[x] = freq.get(x,0) + 1
    return freq


# (end of helper code)
# -----------------------------------

#
# Problem #1: Scoring a word
#
def getWordScore(word, n):
    """
    Returns the score for a word. Assumes the word is a valid word.

    The score for a word is the sum of the points for letters in the
    word, multiplied by the length of the word, PLUS 50 points if all n
    letters are used on the first turn.

    Letters are scored as in Scrabble; A is worth 1, B is worth 3, C is
    worth 3, D is worth 2, E is worth 1, and so on (see SCRABBLE_LETTER_VALUES)

    word: string (lowercase letters)
    n: integer (HAND_SIZE; i.e., hand size required for additional points)
    returns: int >= 0
    """
    value = sum((map(lambda x: SCRABBLE_LETTER_VALUES[x] if x in SCRABBLE_LETTER_VALUES.keys() else 0, word)))
    value *= len(word)
    if len(word) == n:
        value += 50
    return value


#
# Problem #2: Make sure you understand how this function works and what it does!
#
def displayHand(hand):
    """
    Displays the letters currently in the hand.

    For example:
    >>> displayHand({'a':1, 'x':2, 'l':3, 'e':1})
    Should print out something like:
       a x x l l l e
    The order of the letters is unimportant.

    hand: dictionary (string -> int)
    """
    for letter in hand.keys():
        for j in range(hand[letter]):
             print(letter,end=" ")       # print all on the same line
    print()                             # print an empty line

#
# Problem #2: Make sure you understand how this function works and what it does!
#
def dealHand(n):
    """
    Returns a random hand containing n lowercase letters.
    At least n/3 the letters in the hand should be VOWELS.

    Hands are represented as dictionaries. The keys are
    letters and the values are the number of times the
    particular letter is repeated in that hand.

    n: int >= 0
    returns: dictionary (string -> int)
    """
    hand={}
    numVowels = n // 3
    
    for i in range(numVowels):
        x = VOWELS[random.randrange(0,len(VOWELS))]
        hand[x] = hand.get(x, 0) + 1
        
    for i in range(numVowels, n):    
        x = CONSONANTS[random.randrange(0,len(CONSONANTS))]
        hand[x] = hand.get(x, 0) + 1
        
    return hand

#
# Problem #2: Update a hand by removing letters
#
def updateHand(hand, word):
    """
    Assumes that 'hand' has all the letters in word.
    In other words, this assumes that however many times
    a letter appears in 'word', 'hand' has at least as
    many of that letter in it. 

    Updates the hand: uses up the letters in the given word
    and returns the new hand, without those letters in it.

    Has no side effects: does not modify hand.

    word: string
    hand: dictionary (string -> int)    
    returns: dictionary (string -> int)
    """
    dictionary = hand.copy()
    [dictionary.update({x:dictionary[x]-1}) for x in word if x in dictionary.keys() and dictionary[x]!=0]
    return dictionary


#
# Problem #3: Test word validity
#
def isValidWord(word, hand, wordList):
    """
    Returns True if word is in the wordList and is entirely
    composed of letters in the hand. Otherwise, returns False.

    Does not mutate hand or wordList.
   
    word: string
    hand: dictionary (string -> int)
    wordList: list of lowercase strings
    """
    dictionary = hand.copy()
    if  word not in wordList or len(word) ==0:
        return False
    
    return all(map(lambda x: True if x in dictionary.keys() and getFrequencyDict(word)[x] <= dictionary[x] else 0  ,word))

#
# Problem #4: Playing a hand
#
def calculateHandlen(hand):
    """ 
    Returns the length (number of letters) in the current hand.
    
    hand: dictionary (string int)
    returns: integer
    """
    return (sum(map(lambda x: x[1],hand.items())))

def playHand(hand, wordList, n):
    """
    Allows the user to play the given hand, as follows:

    * The hand is displayed.
    * The user may input a word or a single period (the string ".") 
      to indicate they're done playing
    * Invalid words are rejected, and a message is displayed asking
      the user to choose another word until they enter a valid word or "."
    * When a valid word is entered, it uses up letters from the hand.
    * After every valid word: the score for that word is displayed,
      the remaining letters in the hand are displayed, and the user
      is asked to input another word.
    * The sum of the word scores is displayed when the hand finishes.
    * The hand finishes when there are no more unused letters or the user
      inputs a "."

      hand: dictionary (string -> int)
      wordList: list of lowercase strings
      n: integer (HAND_SIZE; i.e., hand size required for additional points)
      
    """
    # BEGIN PSEUDOCODE <-- Remove this comment when you code this function; do your coding within the pseudocode
    #(leaving those comments in-place!)
    # Keep track of the total score
    
    # As long as there are still letters left in the hand:
    total = 0
    while calculateHandlen(hand) != 0:
        
        # Display the hand
        displayHand(hand)
        # Ask user for input
        letter = input('Enter word, or a "." to indicate that you are finished: ')
        # If the input is a single period:
        if letter == '.':
            # End the game (break out of the loop)
            break
            # End the game (break out of the loop)
        elif not isValidWord(letter, hand, wordList):        
        # Otherwise (the input is not a single period):
          
            # If the word is not valid:
                print('\n'+'Invalid word, please try again.')
                # Reject invalid word (print a message followed by a blank line)

            # Otherwise (the word is valid):
        else:
                total += getWordScore(letter, n)
                print('\n'+letter+' earned '+str(getWordScore(letter, n))+' points. Total: '+str(total)+' points.')
             # Update the hand 
                hand = updateHand(hand, letter)

    # Game is over (user entered a '.' or ran out of letters), so tell user the total score
    print('Run out of letters. Total score: '+str(total)+' points.')
                

   



# Problem #5: Playing a game
# 

def playGame(wordList):
    """
    Allow the user to play an arbitrary number of hands.
 
    1) Asks the user to input 'n' or 'r' or 'e'.
      * If the user inputs 'n', let the user play a new (random) hand.
      * If the user inputs 'r', let the user play the last hand again.
      * If the user inputs 'e', exit the game.
      * If the user inputs anything else, tell them their input was invalid.
 
    2) When done playing the hand, repeat from step 1
    """
        # As long as there are still letters left in the hand:
    hand = {}
    while True:
        letter = input('Enter letter n or r or e')
        if letter == 'n':
            hand = dealHand(HAND_SIZE)
            playHand(hand, wordList, HAND_SIZE)
        elif letter == 'r' : 
            if len(hand) == 0:
                print ('You have not played a hand yet. Please play a new hand first!')
            else:
                playHand(hand, wordList, HAND_SIZE)
        elif letter == 'e' :
            break 
        else:
            print ('Invalid command.')

   




# Build data structures used for entire session and play game
#
if __name__ == '__main__':
    wordList = loadWords()
    playGame(wordList)


Loading word list from file...
   83667 words loaded.


In [8]:
def getWordScore(word, n):
    """
    Returns the score for a word. Assumes the word is a valid word.

    The score for a word is the sum of the points for letters in the
    word, multiplied by the length of the word, PLUS 50 points if all n
    letters are used on the first turn.

    Letters are scored as in Scrabble; A is worth 1, B is worth 3, C is
    worth 3, D is worth 2, E is worth 1, and so on (see SCRABBLE_LETTER_VALUES)

    word: string (lowercase letters)
    n: integer (HAND_SIZE; i.e., hand size required for additional points)
    returns: int >= 0
    """
    value = list((map(lambda x: SCRABBLE_LETTER_VALUES[x]-1 if x in SCRABBLE_LETTER_VALUES.keys() else 0, word)))
    value *= len(word)
    if len(word) == n:
        value += 50
    return value



In [9]:
def updateHand(hand, word):
    """
    Assumes that 'hand' has all the letters in word.
    In other words, this assumes that however many times
    a letter appears in 'word', 'hand' has at least as
    many of that letter in it. 

    Updates the hand: uses up the letters in the given word
    and returns the new hand, without those letters in it.

    Has no side effects: does not modify hand.
.update( {'before' : 23} )

list((map(lambda x: dictionary[x] if x in dictionary.keys() else 0, search_word)))

    word: string
    hand: dictionary (string -> int)    
    returns: dictionary (string -> int)
    """
    dictionary = hand.copy()
    [dictionary.update({x:dictionary[x]-1}) for x in word if x in dictionary.keys() and dictionary[x]!=0 ]
 
    return  dictionary
     

In [10]:
def updateHand(hand, word):
    """
    Assumes that 'hand' has all the letters in word.
    In other words, this assumes that however many times
    a letter appears in 'word', 'hand' has at least as
    many of that letter in it. 

    Updates the hand: uses up the letters in the given word
    and returns the new hand, without those letters in it.

    Has no side effects: does not modify hand.
.update( {'before' : 23} )

list((map(lambda x: dictionary[x] if x in dictionary.keys() else 0, search_word)))

    word: string
    hand: dictionary (string -> int)    
    returns: dictionary (string -> int)
    """
    dictionary = hand.copy()
    for x in word :    
        if x in dictionary.keys() and  dictionary[x]!=0:
            dictionary.update({x:dictionary[x]-1}) 
    return dictionary
       

In [11]:
def isValidWord(word, hand, wordList):
    """
    Returns True if word is in the wordList and is entirely
    composed of letters in the hand. Otherwise, returns False.

    Does not mutate hand or wordList.
   
    word: string
    hand: dictionary (string -> int)
    wordList: list of lowercase strings
    """
    dictionary = hand.copy()
    if  word not in wordList or len(word) ==0:
        return False
    return all(map(lambda x: True if x in dictionary.keys() and getFrequencyDict(word)[x] <= dictionary[x] else 0 ,word))

In [12]:
def calculateHandlen(hand):
    """ 
    Returns the length (number of letters) in the current hand.
    
    hand: dictionary (string int)
    returns: integer
    """
   # print({key: len(value) for key, value in hand.items()})
   
    return (sum(map(lambda x: x[1],hand.items())))

calculateHandlen({'b': 1, 'c': 0, 'a': 1})

2

In [13]:
def playHand(hand, wordList, n):
    """
    Allows the user to play the given hand, as follows:

    * The hand is displayed.
    * The user may input a word or a single period (the string ".") 
      to indicate they're done playing
    * Invalid words are rejected, and a message is displayed asking
      the user to choose another word until they enter a valid word or "."
    * When a valid word is entered, it uses up letters from the hand.
    * After every valid word: the score for that word is displayed,
      the remaining letters in the hand are displayed, and the user
      is asked to input another word.
    * The sum of the word scores is displayed when the hand finishes.
    * The hand finishes when there are no more unused letters or the user
      inputs a "."

      hand: dictionary (string -> int)
      wordList: list of lowercase strings
      n: integer (HAND_SIZE; i.e., hand size required for additional points)
      
    """
    # BEGIN PSEUDOCODE <-- Remove this comment when you code this function; do your coding within the pseudocode
    #(leaving those comments in-place!)
    # Keep track of the total score
    
    # As long as there are still letters left in the hand:
    total = 0
    while calculateHandlen(hand) != 0:
        
        # Display the hand
        displayHand(hand)
        # Ask user for input
        letter = input('Enter word, or a "." to indicate that you are finished: ')
        # If the input is a single period:
        if letter == '.':
            # End the game (break out of the loop)
            break
            # End the game (break out of the loop)
        elif not isValidWord(letter, hand, wordList):        
        # Otherwise (the input is not a single period):
          
            # If the word is not valid:
                print('\n'+'Invalid word, please try again.')
                # Reject invalid word (print a message followed by a blank line)

            # Otherwise (the word is valid):
        else:
                total += getWordScore(letter, n)
                print('\n'+letter+' earned '+str(getWordScore(letter, n))+' points. Total: '+str(total)+' points.')
             # Update the hand 
                hand = updateHand(hand, letter)

    # Game is over (user entered a '.' or ran out of letters), so tell user the total score
    print('Run out of letters. Total score: '+str(total)+' points.')
                

   
print(playHand({'a': 1, 'e': 1, 'h': 1, 'm': 2, 'r': 1},wordList, 6))


a e h m m r 
Enter word, or a "." to indicate that you are finished: .


KeyboardInterrupt: 

In [ ]:
def playGame(wordList):
    """
    Allow the user to play an arbitrary number of hands.
 
    1) Asks the user to input 'n' or 'r' or 'e'.
      * If the user inputs 'n', let the user play a new (random) hand.
      * If the user inputs 'r', let the user play the last hand again.
      * If the user inputs 'e', exit the game.
      * If the user inputs anything else, tell them their input was invalid.
 
    2) When done playing the hand, repeat from step 1
    """
    hand = {}
    while True:
        letter = input('Enter letter n or r or e')
        if letter == 'n':
            hand = dealHand(HAND_SIZE)
            playHand(hand, wordList, HAND_SIZE)
        elif letter == 'r' : 
            if len(hand) == 0:
                print ('You have not played a hand yet. Please play a new hand first!')
            else:
                playHand(hand, wordList, HAND_SIZE)
        elif letter == 'e' :
            break 
        else:
            print ('Invalid command.')

   


In [ ]:
def playGame(wordList):
    """
    Allow the user to play an arbitrary number of hands.
 
    1) Asks the user to input 'n' or 'r' or 'e'.
        * If the user inputs 'e', immediately exit the game.
        * If the user inputs anything that's not 'n', 'r', or 'e', keep asking them again.

    2) Asks the user to input a 'u' or a 'c'.
        * If the user inputs anything that's not 'c' or 'u', keep asking them again.

    3) Switch functionality based on the above choices:
        * If the user inputted 'n', play a new (random) hand.
        * Else, if the user inputted 'r', play the last hand again.
          But if no hand was played, output "You have not played a hand yet. 
          Please play a new hand first!"
        
        * If the user inputted 'u', let the user play the game
          with the selected hand, using playHand.
        * If the user inputted 'c', let the computer play the 
          game with the selected hand, using compPlayHand.

    4) After the computer or user has played the hand, repeat from step 1

    wordList: list (string)
    """
    hand = {}
    
    while True:
        
        letter = input("Enter n to deal a new hand, r to replay the last hand, or e to end game:")
      
        if letter not in ['n','r','e']:
            print('Invalid letter')
            continue
        else:
            if letter == 'n':
                hand = dealHand(HAND_SIZE)
                playHand(hand, wordList, HAND_SIZE)
            elif letter == 'r' : 
                if len(hand) == 0:
                    print ('You have not played a hand yet. Please play a new hand first!')
                else:
                    playHand(hand, wordList, HAND_SIZE)
            elif letter == 'c':
                compPlayHand(hand, wordList, n)
                #playHand(hand, wordList, HAND_SIZE)
            elif letter == 'e' :
                break 
            else:
                print ('Invalid command.')
 

In [ ]:
def playGame(wordList):
    """
    Allow the user to play an arbitrary number of hands.
 
    1) Asks the user to input 'n' or 'r' or 'e'.
        * If the user inputs 'e', immediately exit the game.
        * If the user inputs anything that's not 'n', 'r', or 'e', keep asking them again.

    2) Asks the user to input a 'u' or a 'c'.
        * If the user inputs anything that's not 'c' or 'u', keep asking them again.

    3) Switch functionality based on the above choices:
        * If the user inputted 'n', play a new (random) hand.
        * Else, if the user inputted 'r', play the last hand again.
          But if no hand was played, output "You have not played a hand yet. 
          Please play a new hand first!"
        
        * If the user inputted 'u', let the user play the game
          with the selected hand, using playHand.
        * If the user inputted 'c', let the computer play the 
          game with the selected hand, using compPlayHand.

    4) After the computer or user has played the hand, repeat from step 1

    wordList: list (string)
    """
    letter= ''
    hand = {}
    while letter != 'e':
        letter = input('Enter n to deal a new hand, r to replay the last hand, or e to end game: ')
       
        if letter not in ['n','r','e']:
            print ("Invalid command.")
            continue
            
        if letter == 'n':
            hand = dealHand(HAND_SIZE)
        elif letter == 'r and len(hand) == 0: 
            print("You have not played a hand yet. Please play a new hand first!"+'\n')
            continue
                
        while letter != 'e':
            player = input('Enter u to have yourself play, c to have the computer play: ')
            if player not in ['u','c']:
                print ("Invalid command.")
                continue
            if player == 'u':
                playHand(hand, wordList, HAND_SIZE)
            elif player == 'c':
                compPlayHand(hand, wordList, HAND_SIZE)
            break

In [18]:
playGame(wordList)

playGame not yet implemented.
